In [27]:
import os
import json
import ast
import pandas as pd
from urllib.parse import urlparse
import numpy as np
from urllib.parse import urlparse
import nltk
nltk.download('punkt')

from tqdm._tqdm_notebook import tqdm_notebook as tqdm
tqdm.pandas()


DATA_DIRECTORY = '../../data/reddit/Article_data_2019/'
candidates = [ "TRUMP", "SANDERS","DELANEY","WARREN","HARRIS","GILLIBRAND","O'ROURKE","KLOBUCHAR","BOOKER",
    "BUTTIGIEG","GABBARD","YANG","INSLEE","HICKENLOOPER","WILLIAMSON","DE LA FUENTE","CASTRO","BIDEN"]
candidates = set([x.lower() for x in candidates])

[nltk_data] Downloading package punkt to /Users/kippy/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


#### Process Reddit Data

In [2]:
PICKLE_NAME = DATA_DIRECTORY + "reddit_2019_06_15_with_article_text.pkl"
df = pd.read_pickle(PICKLE_NAME)

In [3]:
# Flatten the news dictionary
clean_df = pd.concat( [df.drop(['article_data'],axis=1), df['article_data'].progress_apply(pd.Series)]
                      ,axis = 1)
clean_df.drop(0, axis = 1, inplace=True)


def df_column_uniquify(df):
    df_columns = df.columns
    new_columns = []
    for item in df_columns:
        counter = 0
        newitem = item
        while newitem in new_columns:
            counter += 1
            newitem = "{}_{}".format(item, counter)
        new_columns.append(newitem)
    df.columns = new_columns
    return df

clean_df = df_column_uniquify(clean_df)

In [8]:
print(clean_df.shape)
print(clean_df[(~clean_df['title_1'].isnull()) & (~clean_df['text'].isnull())].shape)
print(clean_df[(~clean_df['title_1'].isnull()) & (~clean_df['text'].isnull())].drop_duplicates('url').shape)

(121986, 22)
(82018, 22)
(64581, 22)


In [9]:
# remove nulls, 
clean_df = clean_df[(~clean_df['title_1'].isnull()) & (~clean_df['text'].isnull())]
clean_df = clean_df.drop_duplicates('url')
clean_df["title_tokenized"] = clean_df.progress_apply(lambda x: [token.lower() for token in nltk.word_tokenize(x.title)]
                                                      , axis = 1)
clean_df["text_tokenized"] = clean_df.progress_apply(lambda x: [token.lower() for token in nltk.word_tokenize(x.text)]
                                                     , axis = 1)
clean_df["domain"] = clean_df.progress_apply(lambda x: urlparse(x.url).netloc, axis = 1)
clean_df['candidate_title'] = clean_df.progress_apply(lambda x: candidates.intersection(x.title_tokenized), axis=1)
clean_df['candidate_text'] = clean_df.progress_apply(lambda x: candidates.intersection(x.text_tokenized), axis=1)

In [46]:
sanders_reddit = clean_df[clean_df.progress_apply(lambda x: 'sanders' in x.candidate_text, axis=1)]
sanders_reddit['count'] = 1
sanders_reddit

/usr/local/Cellar/ipython/7.4.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,id,num_comments,score,subreddit,subreddit_subscribers,title,url,authors,date_download,date_modify,...,title_1,title_page,title_rss,url_1,title_tokenized,text_tokenized,domain,candidate_title,candidate_text,count
7,abifrd,89,1,SandersForPresident,215692,Elizabeth Warren and other Democrats failing o...,https://www.theguardian.com/us-news/2015/apr/2...,[Peter Moskowitz],2019-06-09 18:46:36,None,...,Elizabeth Warren and other Democrats failing o...,NaN,NaN,https://www.theguardian.com/us-news/2015/apr/2...,"[elizabeth, warren, and, other, democrats, fai...","[report, from, liberal, super, pac, calls, out...",www.theguardian.com,{warren},"{warren, sanders}",1
9,abk59b,0,1,SandersForPresident,215682,"A Bold, Progressive Agenda for a Happier and H...",https://www.commondreams.org/views/2019/01/01/...,[Bernie Sanders],2019-06-09 18:51:51,None,...,"A Bold, Progressive Agenda for a Happier and H...",NaN,NaN,https://www.commondreams.org/views/2019/01/01/...,"[a, bold, ,, progressive, agenda, for, a, happ...","[the, following, is, a, new, year, 's, day, em...",www.commondreams.org,{},"{trump, sanders}",1
11,abke9s,375,1,SandersForPresident,215678,Who Wants Bernie to Run?,https://www.jacobinmag.com/2019/01/bernie-sand...,"[Carl Beijer, Luke Savage, Peter Gowan, Meagan...",2019-06-09 18:49:21,None,...,Who Wants Bernie to Run?,NaN,NaN,https://www.jacobinmag.com/2019/01/bernie-sand...,"[who, wants, bernie, to, run, ?]","[how, would, you, feel, if, bernie, sanders, r...",www.jacobinmag.com,{},"{sanders, klobuchar}",1
13,ablqm6,1,1,SandersForPresident,215672,Who wants Bernie to run?,http://jacobinmag.com/2019/01/bernie-sanders-r...,"[Carl Beijer, Luke Savage, Peter Gowan, Meagan...",2019-06-09 18:51:40,None,...,Who Wants Bernie to Run?,NaN,NaN,http://jacobinmag.com/2019/01/bernie-sanders-r...,"[who, wants, bernie, to, run, ?]","[how, would, you, feel, if, bernie, sanders, r...",jacobinmag.com,{},"{sanders, klobuchar}",1
15,abo6f2,3,1,SandersForPresident,215665,Here's how Elizabeth Warren fares in a new 202...,https://www.boston.com/news/politics/2018/08/2...,"[Nik Decosta-Klipa, August]",2019-06-09 18:58:08,None,...,Here's how Elizabeth Warren fares in a new 202...,NaN,NaN,https://www.boston.com/news/politics/2018/08/2...,"[here, 's, how, elizabeth, warren, fares, in, ...","[sen., elizabeth, warren, might, be, the, so-c...",www.boston.com,"{warren, trump}","{warren, biden, trump, sanders}",1
23,abvtp5,241,1,SandersForPresident,215676,Bernie Sanders Isn’t Just Another White Male C...,https://forward.com/opinion/416905/bernie-sand...,"[David Klion, Getty]",2019-06-09 19:10:12,None,...,Bernie Sanders Isn’t Just Another White Male C...,NaN,NaN,https://forward.com/opinion/416905/bernie-sand...,"[bernie, sanders, isn, ’, t, just, another, wh...","[the, 2020, democratic, primary, will, likely,...",forward.com,{sanders},"{biden, klobuchar, gillibrand, trump, booker, ...",1
26,abx6de,9,1,SandersForPresident,215682,Nancy Pelosi Rams Austerity Provision Into Hou...,https://theintercept.com/2019/01/02/nancy-pelo...,[Ryan Grim],2019-06-09 19:06:15,None,...,Nancy Pelosi Rams Austerity Provision Into Hou...,NaN,NaN,https://theintercept.com/2019/01/02/nancy-pelo...,"[nancy, pelosi, rams, austerity, provision, in...","[“, there, ’, s, enormous, appetite, in, the, ...",theintercept.com,{},"{warren, trump, sanders}",1
30,abx75r,0,1,SandersForPresident,215682,2019–The Year of Medicare for All,https://www.commondreams.org/views/2019/01/02/...,[Zenei Cortez],2019-06-09 19:12:55,None,...,2019: The Year of Medicare for All,NaN,NaN,https://www.commondreams.org/views/2019/01/02/...,"[2019–the, year, of, medicare, for, all]","[early, in, this, new, year, ,, rep., pramilla...",www.commondreams.org,{},"{trump, sanders}",1
31,aby7ps,25,1,SandersForPresident,215678,"For Bernie Sanders, Claims of Sexism in 2016 C...",https://www.nytimes.com/2019/01/02/us/politics...,"[Sydney Ember, Katie Benner]",2019-06-09 19:35:12,None,...,Sexism Claims From Bernie Sanders’s 2016 Run: ...,

Donations Data

In [29]:
# find the path to each fec file, store paths in a nested dict
fec_2020_paths = {}
base_path = os.path.join("../../","data","fec","2020")
for party_dir in os.listdir(base_path):
    if(party_dir[0]!="."):
        fec_2020_paths[party_dir] = {}
        for cand_dir in os.listdir(os.path.join(base_path,party_dir)):
            if(cand_dir[0]!="."):
                fec_2020_paths[party_dir][cand_dir] = {}
                for csv_path in os.listdir(os.path.join(base_path,party_dir,cand_dir)):
                    if(csv_path.find("schedule_a")>=0):
                        fec_2020_paths[party_dir][cand_dir]["donations"] = \
                            os.path.join(base_path,party_dir,cand_dir,csv_path)
                    elif(csv_path.find("schedule_b")>=0):
                        fec_2020_paths[party_dir][cand_dir]["spending"] = \
                            os.path.join(base_path,party_dir,cand_dir,csv_path)

In [30]:
candid = 'sanders'
if("donations" in fec_2020_paths["democrat"][candid].keys()):
        donate_df = pd.read_csv(fec_2020_paths["democrat"][candid]["donations"])
        donate_df["contribution_receipt_date"] = pd.to_datetime(donate_df["contribution_receipt_date"])
        ts = donate_df.groupby(by="contribution_receipt_date")["contribution_receipt_amount"].sum()

/usr/local/Cellar/ipython/7.4.0/libexec/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (35) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [66]:
ts

contribution_receipt_date
2019-02-19     781568.23
2019-02-20    1250053.37
2019-02-21     197539.69
2019-02-22      51822.70
2019-02-23      27343.05
2019-02-24     131872.00
2019-02-25     110597.38
2019-02-26      36369.74
2019-02-27    1027014.60
2019-02-28      30492.12
2019-03-01      15546.89
2019-03-02      31127.58
2019-03-03      70904.29
2019-03-04      48379.43
2019-03-05      24130.99
2019-03-06      18225.76
2019-03-07      55186.17
2019-03-08      13605.59
2019-03-09      26302.85
2019-03-10      66513.15
2019-03-11      37985.42
2019-03-12      25476.53
2019-03-13      23515.07
2019-03-14      93526.28
2019-03-15       2993.98
2019-03-16        658.28
2019-03-17     119547.50
2019-03-18     142198.87
2019-03-19      98038.01
2019-03-20      90945.38
2019-03-21      49854.89
2019-03-22      11652.04
2019-03-23      11388.69
2019-03-24      39951.78
2019-03-25     137046.42
2019-03-26     302654.17
2019-03-27     235093.41
2019-03-28     200897.81
2019-03-29     140644.56

In [58]:
sanders_reddit.columns
sanders_reddit['num_comments'].sum()

530038

In [65]:
#sanders_reddit[['score', 'date_publish', 'num_comments', 'subreddit']].sort_values(['date_publish'])
sanders_reddit.fillna(0)[['date_publish', 'score', 'num_comments', 'count']].groupby(['date_publish']).agg('sum')

,score,num_comments,count
date_publish,,,
1981-03-08 00:00:00,31,1,1
1998-12-18 00:00:00,1,1,1
2001-03-19 00:00:00,68,54,3
2001-04-19 00:00:00,738,114,2
2001-05-19 00:00:00,635,31,2
2002-04-19 00:00:00,1064,82,4
2002-05-19 00:00:00,32428,3412,4
2002-06-19 00:00:00,135,78,3
2002-11-09 11:25:09,1,1,1
